In [ ]:
from torch import nn

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, embed_dim, input_dim, dropout=0.1):
        super().__init__()

        self.embed_dim = embed_dim
        self.heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.qkv_lin = nn.Linear(input_dim, embed_dim*3)    # bias=False?
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = dropout


    def forward(self, qkv, mask=None, dropout=None):
        batch_size, seq_length, embed_dim = qkv.size()
        num_heads, head_dim = self.num_heads, self.head_dim

        qkv = self.qkv_lin(qkv)
        qkv = qkv.reshape(batch_size, seq_length, num_heads, head_dim).permute(0, 2, 1, 3)
        q,k,v = qkv.chunk(3, -1)

        # NOTE: no mask parameter??
        scores = attention(q, k, v, head_dim, dropout) #mask ?
        
        scores = scores.permute(0, 2, 1, 3).reshape(batch_size, seq_length, embed_dim)

        output = self.out_proj(scores)

        return output


# DUMMY FUNCTION --> this is qingyuan's function
def attention(q, k, v, mask=None):
    return 0




        


